In [149]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

In [7]:
path = "raw/par3/par3.pkl"
with open(path, 'rb') as f:
    data = pd.read_pickle(f)

In [13]:
keys = list(data.keys())

In [18]:
data[keys[0]].keys()

dict_keys(['source_paras', 'gt_paras', 'translator_data'])

## Goal

I want to construct a dataset of paragraphs aligned with translations.
Book is irrelevant. I do not want google translate version - I can just ignore these.

Structure should be [source-text, translation-alignment]

### Counting the number of translators for a given book

In [42]:
def find_book_with_n_translators(book_data, n_translations):
    books = [book for book in keys if len(book_data[book]["translator_data"].keys()) > n_translations]
    return books

In [58]:
n = 2
n_translator_books = find_book_with_n_translators(data, n)

In [59]:
number_of_paragraphs = 0 
for book in n_translator_books:
    for i in range(n):
        number_of_paragraphs += len(data[book]["translator_data"][f"translator_{i+1}"]["translator_paras"])
print(number_of_paragraphs)

65372


## Making two datasets - 3 Translations & 2 Translations

### 3 Translations first

In [152]:
n = 2
data_dict = {f"translation_{i+1}":[] for i in range(n)}
n_translator_books = find_book_with_n_translators(data, n)
for book in n_translator_books:
    chosen_book = data[book]
    translated_paragraphs = []
    for i in range(n):
        trans_paragraphs = chosen_book["translator_data"][f"translator_{i+1}"]["translator_paras"]
        data_dict[f"translation_{i+1}"] = data_dict.get(f"translation_{i+1}") + trans_paragraphs
two_translation_df = pd.DataFrame(data_dict, columns=[f"translation_{i+1}" for i in range(n)])

df_mask = np.ones(shape=two_translation_df.shape, dtype=int)
for i, col in enumerate(two_translation_df.columns):
    row_mask = two_translation_df[col].str.split().apply(lambda x: len(x) > 10)
    df_mask[:, i] = row_mask
df_mask = np.prod(df_mask, axis=1) % 2 == 1 # Only keep the values which have more than 10 words.

two_translation_df = two_translation_df[df_mask]

In [157]:
# train_validation, test = train_test_split(two_translation_df, test_size=0.2)
# train, validation = train_test_split(train_validation, test_size=0.25)
# 
# train.to_csv("prepared/train/par3/two_trans_train_untokenized.csv", index=False)
# validation.to_csv("prepared/validation/par3/two_trans_validation_untokenized.csv", index=False)
# test.to_csv("prepared/test/par3/two_trans_test_par3_untokeznied.csv", index=False)